In [16]:
import numpy as np
import json
import re
import tensorflow as tf
import random
import spacy
nlp = spacy.load('en_core_web_sm')


In [17]:
with open('intent.json') as f:
    intents = json.load(f)


In [18]:
def preprocessing(line):
    line = re.sub(r'[^a-zA-z.?!\']', ' ', line)
    line = re.sub(r'[ ]+', ' ', line)
    return line


In [20]:
# get text and intent title from json data
inputs, targets = [], []
classes = []
intent_doc = {}

for intent in intents['intents']:
    if intent['intent'] not in classes:
        classes.append(intent['intent'])
    if intent['intent'] not in intent_doc:
        intent_doc[intent['intent']] = []
        
    for text in intent['text']:
        inputs.append(preprocessing(text))
        targets.append(intent['intent'])
        
    for response in intent['responses']:
        intent_doc[intent['intent']].append(response)

In [21]:
print(intent)

{'intent': 'SelfAware', 'text': ['Can you prove you are self-aware', 'Can you prove you are self aware', 'Can you prove you have a conscious', 'Can you prove you are self-aware please', 'Can you prove you are self aware please', 'Can you prove you have a conscious please', 'prove you have a conscious'], 'responses': ['That is an interesting question, can you prove that you are?', 'That is an difficult question, can you prove that you are?', 'That depends, can you prove that you are?'], 'extension': {'function': '', 'entities': False, 'responses': []}, 'context': {'in': '', 'out': '', 'clear': False}, 'entityType': 'NA', 'entities': []}


In [22]:
def tokenize_data(input_list):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>')
    
    tokenizer.fit_on_texts(input_list)
    
    input_seq = tokenizer.texts_to_sequences(input_list)

    input_seq = tf.keras.preprocessing.sequence.pad_sequences(input_seq, padding='pre')
    
    return tokenizer, input_seq

# preprocess input data
tokenizer, input_tensor = tokenize_data(inputs)


In [23]:
def create_categorical_target(targets):
    word={}
    categorical_target=[]
    counter=0
    for trg in targets:
        if trg not in word:
            word[trg]=counter
            counter+=1
        categorical_target.append(word[trg])
    
    categorical_tensor = tf.keras.utils.to_categorical(categorical_target, num_classes=len(word), dtype='int32')
    return categorical_tensor, dict((v,k) for k, v in word.items())

# preprocess output data
target_tensor, trg_index_word = create_categorical_target(targets)


In [24]:
print('input shape: {} and output shape: {}'.format(input_tensor.shape, target_tensor.shape))


input shape: (243, 9) and output shape: (243, 77)


In [25]:
# hyperparameters
epochs=50
vocab_size=len(tokenizer.word_index) + 1
embed_dim=512
units=128
target_length=target_tensor.shape[1]


In [26]:
# build RNN Model with tensorflow
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embed_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units, dropout=0.2)),
    tf.keras.layers.Dense(units, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(target_length, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(lr=1e-2)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 512)         112128    
                                                                 
 bidirectional (Bidirectiona  (None, 256)              656384    
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 77)                9933      
                                                                 
Total params: 811,341
Trainable params: 811,341
Non-trainable params: 0
__________________________________________________

E:\Programs\Anaconda\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [27]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)

# train the model
model.fit(input_tensor, target_tensor, epochs=epochs, callbacks=[early_stop])


Epoch 1/50
8/8 [==============================] - 4s 43ms/step - loss: 4.3232 - accuracy: 0.0494
Epoch 2/50
8/8 [==============================] - 0s 41ms/step - loss: 3.6523 - accuracy: 0.1440
Epoch 3/50
8/8 [==============================] - 0s 41ms/step - loss: 3.0194 - accuracy: 0.2922
Epoch 4/50
8/8 [==============================] - 0s 41ms/step - loss: 2.2398 - accuracy: 0.4444
Epoch 5/50
8/8 [==============================] - 0s 45ms/step - loss: 1.7106 - accuracy: 0.5761
Epoch 6/50
8/8 [==============================] - 0s 43ms/step - loss: 1.3534 - accuracy: 0.6584
Epoch 7/50
8/8 [==============================] - 0s 42ms/step - loss: 0.9993 - accuracy: 0.7202
Epoch 8/50
8/8 [==============================] - 0s 43ms/step - loss: 0.6800 - accuracy: 0.8107
Epoch 9/50
8/8 [==============================] - 0s 41ms/step - loss: 0.6790 - accuracy: 0.7778
Epoch 10/50
8/8 [==============================] - 0s 41ms/step - loss: 0.5521 - accuracy: 0.8189
Epoch 11/50
8/8 [============

In [28]:
model.save('saved_model/my_model2.h5')


In [29]:
new_model = tf.keras.models.load_model('saved_model/my_model2.h5')

# Check its architecture
new_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 512)         112128    
                                                                 
 bidirectional (Bidirectiona  (None, 256)              656384    
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 77)                9933      
                                                                 
Total params: 811,341
Trainable params: 811,341
Non-trainable params: 0
__________________________________________________

In [31]:
def response(sentence):
    sent_seq = []
    doc = nlp(repr(sentence))
    
    # split the input sentences into words
    for token in doc:
        if token.text in tokenizer.word_index:
            sent_seq.append(tokenizer.word_index[token.text])

        # handle the unknown words error
        else:
            sent_seq.append(tokenizer.word_index['<unk>'])

    sent_seq = tf.expand_dims(sent_seq, 0)
    # predict the category of input sentences
    pred = new_model(sent_seq)

    pred_class = np.argmax(pred.numpy(), axis=1)
    
    # choice a random response for predicted sentence
    return random.choice(intent_doc[trg_index_word[pred_class[0]]]), trg_index_word[pred_class[0]]

# chat with bot
print("Note: Enter 'quit' to break the loop.")
while True:
    input_ = input('You: ')
    if input_.lower() == 'quit':
        break
    res, typ = response(input_)
    print('Bot: {} -- TYPE: {}'.format(res, typ))
    print()


Note: Enter 'quit' to break the loop.
Bot: Yes. I work and all my operations are based on the internet servers. -- TYPE: bot1

Bot: There are currently no upcoming events -- TYPE: events

Bot:  Python. -- TYPE: lang

Bot:  I am written in Python. -- TYPE: lang

Bot:  I am written in Python. -- TYPE: lang

Bot: There are currently no upcoming events -- TYPE: events

Bot:  Python. -- TYPE: lang

Bot:  I am written in Python. -- TYPE: lang

Bot: hello thanks for checking in -- TYPE: greetings

Bot: Man: Doctor, I've just swallowed a pillow. Doctor: How do you feel? Man: A little down in the mouth. -- TYPE: Jokes

